In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
%matplotlib inline

In [2]:
# Obtenemos los archivos de contaminación de la comunidad de Madrid.
fileList = ! ls ./Madrid/
fileList

['datos2001.txt',
 'datos2002.txt',
 'datos2003.txt',
 'datos2004.txt',
 'datos2005.txt',
 'datos2006.txt',
 'datos2007.txt',
 'datos2008.txt',
 'datos2009.txt',
 'datos2010.txt',
 'datos2011.txt',
 'datos2012.txt',
 'datos2013.txt',
 'datos2014.txt',
 'datos2015.txt',
 'datos2016.txt',
 'datos2017.txt']

In [51]:
# Creamos los nombres de las columnas.
columns=['ESTACION', 'MAGNITUD', 'TECNICA', 'DATO HORARIO', 'ANIO', 'MES', 'DIA 1', 'DIA 2', 'DIA 3', 'DIA 4']
columns2010=['ESTACION', 'MAGNITUD', 'TECNICA', 'DATO HORARIO', 'ANIO', 'MES', 'DEL', 'DIA 1', 'DIA 2', 'DIA 3', 'DIA 4']

# Creamos el DataFrame padre, donde se concatenarán los demás.
data = pd.DataFrame(columns=columns)

# Creamos un acumulador porque hasta el año 2010 se utiliza una codificación y a partir de ahí, otra distinta.
acc=0

# Añadimos todos los archivos en un mismo dataFrame para poder manipularlo.
for file_txt in fileList:
    
    # En caso de fichero anterior a 2010 o del 2010.
    if(acc < 10):
        table = pd.read_fwf('./Madrid/'+file_txt, widths=[8,2,2,2,2,2,2,6,6,6,6], header=None, nwors = 3, names=columns2010)
        table = table.drop('DEL', 1)
    # En caso de fichero moderno.
    else:
        table = pd.read_fwf('./Madrid/'+file_txt, widths=[8,2,2,2,2,2,6,6,6,6], header=None, nwors = 3, names=columns)
    acc=acc+1
    data =pd.concat([data,table])
    
data.sort_values(['ANIO', 'MES'], ascending=True)

,ESTACION,MAGNITUD,TECNICA,DATO HORARIO,ANIO,MES,DIA 1,DIA 2,DIA 3,DIA 4
0,28079001,1,38,4,1,1,00.00N,00.00N,16.31V,17.04V
12,28079001,6,48,4,1,1,00.00N,00.00N,1.250V,1.160V
24,28079001,8,8,4,1,1,00.00N,00.00N,80.17V,76.12V
36,28079001,10,47,4,1,1,00.00N,00.00N,37.37V,35.88V
48,28079001,12,8,4,1,1,00.00N,00.00N,174.6V,158.2V
60,28079001,14,6,4,1,1,00.00N,00.00N,10.85V,11.89V
72,28079002,1,38,4,1,1,13.03V,18.29V,22.15V,21.31V
84,28079002,6,48,4,1,1,1.150V,1.070V,1.400V,1.410V
96,28079002,8,8,4,1,1,68.75V,84.79V,86.60V,83.71V
108,28079002,10,47,4,1,1,21.82V,31.31V,42.91V,45.43V


In [52]:
len(data)

34606

# Filtrado de datos
### Según el dosier facilitado por el ayuntamiento de Madrid, todos los datos que no contengan en su último carácter una 'V', no podrán ser considerados verificados.

### 1- Primero hay que decidir que mágnitudes queremos medir para extraerlas y analizarlas.


### 2- Una vez tengamos los datos filtrados, hay que formatear el dataFrame

In [30]:

# Para los que no son validados, el primer día, se les igualará al segundo y en el último día con el antepenúltimo.
# Los días entre medias, serán la media de el día anterior y posterior.
# Además, se eliminará la 'V' y se convertirá a los números en Interger.
for d in data.index:
    if data['DIA 1'][d][-1:].lower() != 'v':
        data['DIA 1'][d]=int(data['DIA 2'][d][:-1])
    elif data['DIA 2'][d][-1:].lower() != 'v':
        data['DIA 2'][d]=(int(data['DIA 1'][d][:-1])+int(data['DIA 3'][d][:-1]))/2
    elif data['DIA 3'][d][-1:].lower() != 'v':
        data['DIA 3'][d]=(int(data['DIA 2'][d][:-1])+int(data['DIA 4'][d][:-1]))/2
    elif data['DIA 4'][d][-1:].lower() != 'v':
        data['DIA 4'][d]=int(data['DIA 3'][d][:-1])
    else: 
        try:
            data['DIA 1'][d]=int(data['DIA 1'][d][:-1])
            data['DIA 2'][d]=int(data['DIA 2'][d][:-1])
            data['DIA 3'][d]=int(data['DIA 3'][d][:-1])
            data['DIA 4'][d]=int(data['DIA 4'][d][:-1])
            break
        except ValueError:
             print("Oops!  That was no valid number.  Try again...")
len(data)

/home/datascience/.local/lib/python2.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/datascience/.local/lib/python2.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/datascience/.local/lib/python2.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/datascience/.local/lib/python2.7/site-packages/ipykernel_launcher.py:19: SettingWithCop

496

In [31]:
data.head(10)

,ESTACION,MAGNITUD,TECNICA,DATO HORARIO,ANIO,MES,DIA 1,DIA 2,DIA 3,DIA 4
0,28079004,1,38,4,17,1,6,8,12,9
1,28079004,1,38,4,17,2,00008V,00004V,00003V,00003V
2,28079004,1,38,4,17,3,00005V,00007V,00005V,00003V
3,28079004,1,38,4,17,4,00004V,00004V,00005V,00007V
4,28079004,6,48,4,17,1,000.5V,000.5V,000.7V,000.7V
5,28079004,6,48,4,17,2,000.6V,000.3V,000.2V,000.2V
6,28079004,6,48,4,17,3,000.3V,000.5V,000.4V,000.2V
7,28079004,6,48,4,17,4,000.3V,000.4V,000.4V,000.6V
8,28079004,7,8,4,17,1,00055V,00059V,00133V,00112V
9,28079004,7,8,4,17,2,00073V,00011V,00003V,00002V


In [25]:
d=2
print((int(data['DIA 1'][d][:-1])+int(data['DIA 1'][d][:-1]))/2)

5
